In [1]:
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import pandas as pd
import torch
import csv
from bert_score import score
import clickhouse_driver

# Данные

In [39]:
test_data= pd.read_csv("data.csv") 
test_data["answer"] = test_data["answer"].str.replace("\n", "")

# Был взят небольшой эмбеддер на базе BERTа, способный работать с русским языком https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2
## При возможности, можно взять эмбеддер побольше - https://huggingface.co/intfloat/e5-mistral-7b-instruct, но мы сочли, что нам хватит и небольшого, лучше взять модель 7B

In [42]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = model.to('cuda')
embeddings = model.encode(test_data['answer'])

[2024-03-14 12:40:27,668] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2024-03-14 12:40:28.222567: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [45]:
embeddings.shape

(611, 768)

# Пайплайн PCA для понижения размерности, 768 - слишком много, 300 - оптимально, если система все равно нагружается сильно, то можно попробовать понизить ещё до ~100-200

In [46]:
pca = Pipeline(steps=[
    ('mean', StandardScaler(with_mean=True, with_std=False)),
    ('pca', PCA(n_components=300, random_state=42)),
    ('std', StandardScaler(with_mean=True, with_std=True))
])

In [47]:
pca.fit(embeddings)
text_vectors = pca.transform(embeddings)
text_vectors.shape

(611, 300)

# Clickhouse

In [48]:
# Создание таблицы
import clickhouse_driver

client = clickhouse_driver.Client(host='localhost')

# client.execute("DROP TABLE embeddings") # Если нужно удалить существующую таблицу
client.execute("""
CREATE TABLE embeddings (
    id UInt32,
    embedding Array(Float32),
    link String
)
ENGINE = MergeTree
PRIMARY KEY (id)
""")


[]

In [66]:
# Проверка существующей таблицы
len(client.execute("SELECT * FROM embeddings"))

611

In [51]:
# Заполнение таблицы
import clickhouse_connect

client = clickhouse_connect.get_client()

embeddings = text_vectors.tolist()
links = test_data['link'].tolist()
ids = list(range(len(text_vectors)))

data = []

for i in range(len(ids)):
    row = [ids[i], embeddings[i], links[i]]
    data.append(row)

client.insert('embeddings', data, column_names=['id', 'embedding', 'link'])

# Загружаем языковую модель и токенизатор

## Была выбрана mistral instruct. Во-первых, модель является оптимальной по количеству парамтров и качеству генерации, так, Mistral 7B побеждает Llama v2 13B на большинстве бенчмарков.      
## Во-вторых, у модели достаточно большой размер контекста - 8К токенов, что позволяет помещать в контекст параграфы большого размера и не терять важную информацию при нарезке документов.      
## В-третьих, instuct версия дополнительно обучена на аннотированном (размеченном) вручную инструкционном датасете (каждая инструкция - это пара "вопрос" - "правильный с точки зрения человека ответ", а обычные, не иструкционные модели, до этой стадии дообучения не дошли, а просто обучены предсказывать следующее слово на очень большом, но неразмеченном текстовом корпусе).   
## Кроме того, модель неплохо работает с русским языком

In [55]:
llm_dirname = '../nsu-ai/team_code/models/llm'

In [ ]:
# Если у вас модель не загружена локально, то
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
# https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2, если недоступна последняя версия transformers, то можно использовать v0.1 - https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1
# если версии библиотек не самые новые, то v0.1 может работать быстрее

In [56]:
llm_model = AutoModelForCausalLM.from_pretrained(llm_dirname, torch_dtype=torch.float16, device_map={"":0}) # device_map требует установки Accelerate
tokenizer = AutoTokenizer.from_pretrained(llm_dirname)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Получение индекса наиболее подходящего документа по запросу (можно переделать на несколько подходящих индексов (размер контекста позволяет)

## Можно возвращать несколько документов, добавлять несколько в контекст (если взять чанки меньшего размера) Также можно добавить небольшую модель, которая без Retrieval будет генерировать ответ, на основе него дополнительно ранжировать топ-N документов по близости эмбеддингов

In [57]:
def get_closest_document_idx(query: list) -> int:
    embedding = model.encode(query)
    pca_embedding = pca.transform(embedding)
    pca_embedding = np.squeeze(pca_embedding)
    query_vector = pca_embedding.tolist()
    results = client.execute("""
    SELECT id, embedding, link
    FROM embeddings
    ORDER BY L2Distance(embedding, {}) ASC
    LIMIT 1
    """.format(query_vector))
    
    return results[0][0]
    

# Генерация ответа моделью

In [73]:
def get_completion(query: str, model, tokenizer, documents=test_data, with_link=False) -> str:
    device = "cuda:0"
    document_idx = get_closest_document_idx([query])
    found_text = documents['answer'][document_idx]
    prompt_template = f"""[INST]Ты русскоязычный помощник банка россии, который отвечает на вопросы. Ты знаешь что: {found_text} Вопрос: {query} Отвечай только то, в чем уверен. [/INST]
    """
    prompt = prompt_template.format(query=query)

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

    model_inputs = encodeds.to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=500)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    if with_link:
        return (decoded[0] + ' Ответ основан на ' + documents['link'][document_idx], len(prompt))
    return (decoded[0], len(prompt))


# Тест

In [80]:
result, l = get_completion(query='Что такое курс рубля и почему он меняется?', model=llm_model, tokenizer=tokenizer, documents=test_data, with_link=True)
result[l:]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Я уверен, что курс рубля определяется соотношением спроса на иностранную валюту и ее предложения на валютном рынке. Причинами изменения курса рубля могут быть любые факторы, влекущие изменение соотношения между спросом на иностранную валюту и ее предложением. В отдельные периоды могут преобладать факторы в пользу ослабления рубля, несмотря на одновременное действие других факторов в сторону его укрепления. Например, в конце 2013 — начале 2014 года интерес международных инвесторов к активам стран с формирующимися рынками, в том числе к российским активам, заметно снизился. Это оказалось более значимым, чем высокие цены на нефть в данный период, что привело к ослаблению рубля. Ответ основан на https://www.cbr.ru/dkp/faq/'

# Подсчет BERT F1 Score - https://github.com/Tiiiger/bert_score/

## первые 333 элемента из раздела вопрос-ответ, поэтому на вопросах оттуда можно проверить работу системы

In [86]:
refs = test_data['answer'][:333].values.tolist()  

## Для простоты возьмем первые 150

In [ ]:
answers = []
for question in test_data['text'][:150]:
    result, l = get_completion(query=question, model=llm_model, tokenizer=tokenizer, documents=test_data)
    answers.append(' '.join(result[l+1:].split()).strip())

In [89]:
P, R, F1 = score(answers[:150], refs[:150], lang='en', verbose=True) # Precision, Recall, F1

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/5 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 7.54 seconds, 19.88 sentences/sec


In [91]:
print('На 150 вопросах из раздела вопрос-ответ, усредненный BERT F1 score: ', F1.mean())

На 150 вопросах из раздела вопрос-ответ, усредненный BERT F1 score:  tensor(0.8951)
